In [1]:
import requests
youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res = requests.get(youbike_url)
if res.status_code == 200:
    print('下載成功')
else:
    print('下載失敗')

下載成功


In [5]:
from pydantic import BaseModel, Field, RootModel, field_validator

class Site(BaseModel):
    站名: str = Field(alias='sna')
    區域: str = Field(alias='sarea')
    時間: str = Field(alias='mday')
    地點: str = Field(alias='ar')
    總數: int = Field(alias='total')
    可借: int = Field(alias='available_rent_bikes')
    可還: int = Field(alias='available_return_bikes')
    經度: float = Field(alias='longitude')
    緯度: float = Field(alias='latitude')
    狀態: bool = Field(alias='act')

    @field_validator('站名')
    @classmethod
    def get_name(cls, value: str) -> str:
        return value.split('_')[-1]

class YouBike(RootModel):
    root: list[Site]
    
data: YouBike = YouBike.model_validate_json(res.text)
all_sites: list[dict] = data.model_dump()

In [6]:
import pandas as pd

df = pd.DataFrame(all_sites)
df

,站名,區域,時間,地點,總數,可借,可還,經度,緯度,狀態
0,捷運科技大樓站,大安區,2024-06-17 19:14:20,復興南路二段235號前,28,3,25,121.54360,25.02605,True
1,復興南路二段273號前,大安區,2024-06-17 19:15:22,復興南路二段273號西側,21,12,9,121.54357,25.02565,True
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,16,2,14,121.54124,25.02429,True
3,和平公園東側,大安區,2024-06-17 19:15:22,和平東路二段118巷33號,11,11,0,121.54282,25.02351,True
4,辛亥復興路口西北側,大安區,2024-06-17 19:15:22,復興南路二段368號,16,9,7,121.54299,25.02153,True
...,...,...,...,...,...,...,...,...,...,...
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:13:15,臺大圖書館西南側,30,0,30,121.54031,25.01690,True
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:15:22,臺大霖澤館南側,20,2,18,121.54347,25.01995,True
1420,臺大獸醫館南側,臺大公館校區,2024-06-17 19:13:50,臺大獸醫系館南側,24,1,23,121.54242,25.01791,True
1421,臺大新體育館東南側,臺大公館校區,2024-06-17 19:13:21,臺大體育館東側,40,13,27,121.53591,25.02112,True


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1423 entries, 0 to 1422
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1423 non-null   object 
 1   區域      1423 non-null   object 
 2   時間      1423 non-null   object 
 3   地點      1423 non-null   object 
 4   總數      1423 non-null   int64  
 5   可借      1423 non-null   int64  
 6   可還      1423 non-null   int64  
 7   經度      1423 non-null   float64
 8   緯度      1423 non-null   float64
 9   狀態      1423 non-null   bool   
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 101.6+ KB


In [11]:
less3_df = df.query('可借 <= 3')
less3_df.shape

(434, 10)

In [14]:
df2 = df.query('可借 <=3 and 可還 >= 30')
df2

,站名,區域,時間,地點,總數,可借,可還,經度,緯度,狀態
21,臺灣科技大學正門,大安區,2024-06-17 19:06:22,基隆路四段43號(臺灣科技大學正門旁小側門靠田徑場),50,2,48,121.54142,25.01451,True
22,臺灣科技大學側門,大安區,2024-06-17 19:11:15,基隆路四段73巷與基隆路口,99,0,99,121.53973,25.01295,True
24,臺灣科技大學後門,大安區,2024-06-17 19:15:22,基隆路四段41巷68弄臺科帆船大樓旁,39,1,36,121.54165,25.01182,True
29,臺大男一舍前,大安區,2024-06-17 19:13:21,長興街50號西北側,61,3,58,121.54535,25.01637,True
47,嘉興公園,大安區,2024-06-17 19:14:19,樂業街101巷3號西北側,44,2,37,121.55211,25.02100,True
...,...,...,...,...,...,...,...,...,...,...
1399,臺大社科院西側,臺大公館校區,2024-06-17 19:15:16,臺大社科院西側,42,0,42,121.54145,25.02053,True
1400,臺大社會系館南側,臺大公館校區,2024-06-17 19:06:22,臺大社會及社工館南側,39,1,38,121.54175,25.01987,True
1405,臺大生命科學館西北側,臺大公館校區,2024-06-17 18:44:16,臺大動物博物館西側,30,0,30,121.53825,25.01548,True
1406,臺大第一活動中心西南側,臺大公館校區,2024-06-17 19:13:15,第一學生活動中心東南側,40,0,40,121.53995,25.01761,True


In [16]:
df2 = df.query('可借 <=3 or 可還 <= 3')
df2

,站名,區域,時間,地點,總數,可借,可還,經度,緯度,狀態
0,捷運科技大樓站,大安區,2024-06-17 19:14:20,復興南路二段235號前,28,3,25,121.54360,25.02605,True
2,國北教大實小東側門,大安區,2024-06-17 19:06:21,和平東路二段96巷7號,16,2,14,121.54124,25.02429,True
3,和平公園東側,大安區,2024-06-17 19:15:22,和平東路二段118巷33號,11,11,0,121.54282,25.02351,True
5,復興南路二段280號前,大安區,2024-06-17 19:05:23,復興南路二段280號,11,10,1,121.54328,25.02429,True
6,復興南路二段340巷口,大安區,2024-06-17 19:13:21,復興南路二段342號,11,8,3,121.54326,25.02253,True
...,...,...,...,...,...,...,...,...,...,...
1415,臺大鄭江樓北側,臺大公館校區,2024-06-17 19:15:16,臺大新聞研究所南側,10,0,10,121.54305,25.01900,True
1416,臺大電機二館東南側,臺大公館校區,2024-06-17 19:12:16,臺大電機二館南側,14,0,14,121.54247,25.01857,True
1418,臺大總圖書館西南側,臺大公館校區,2024-06-17 19:13:15,臺大圖書館西南側,30,0,30,121.54031,25.01690,True
1419,臺大黑森林西側,臺大公館校區,2024-06-17 19:15:22,臺大霖澤館南側,20,2,18,121.54347,25.01995,True


In [18]:
df3 = df.query('區域 == "大安區" and 可借 == 0')
df3

,站名,區域,時間,地點,總數,可借,可還,經度,緯度,狀態
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,5,0,5,121.53310,25.02093,False
22,臺灣科技大學側門,大安區,2024-06-17 19:11:15,基隆路四段73巷與基隆路口,99,0,99,121.53973,25.01295,True
25,臺大醫學院附設癌醫中心,大安區,2024-06-17 19:15:22,基隆路三段155巷57號西側,20,0,18,121.54382,25.01426,True
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,18,0,18,121.54291,25.01576,True
67,敦化南路二段293巷口南側,大安區,2024-06-17 18:58:15,敦化南路二段293巷口南側,14,0,14,121.54899,25.02370,True
69,安和路二段181巷口,大安區,2024-06-17 19:14:20,安和路二段185號,17,0,17,121.54988,25.02721,True
72,台北科技大學億光大樓,大安區,2024-06-17 18:58:15,忠孝東路三段81號,45,0,45,121.53833,25.04177,True
76,復興南路1段340巷口,大安區,2024-06-17 19:09:14,復興南路一段信義路三段147巷45弄口,50,0,50,121.54348,25.03701,True
88,昌隆公園,大安區,2024-06-17 19:15:16,市民大道三段206號西側,16,0,16,121.53946,25.04448,True
98,瑠公公園,大安區,2024-03-30 08:59:14,忠孝東路四段49巷4弄/大安路口,20,0,20,121.54612,25.04201,False


In [19]:
n = 0
df4 = df.query('區域 == "大安區" and 可借 == @n')
df4

,站名,區域,時間,地點,總數,可借,可還,經度,緯度,狀態
17,溫州停車場,大安區,2024-06-03 14:08:52,新生南路三段54巷/溫州街口,5,0,5,121.53310,25.02093,False
22,臺灣科技大學側門,大安區,2024-06-17 19:11:15,基隆路四段73巷與基隆路口,99,0,99,121.53973,25.01295,True
25,臺大醫學院附設癌醫中心,大安區,2024-06-17 19:15:22,基隆路三段155巷57號西側,20,0,18,121.54382,25.01426,True
31,臺大動物醫院前,大安區,2024-06-17 19:13:15,基隆路三段155巷/基隆路四段口,18,0,18,121.54291,25.01576,True
67,敦化南路二段293巷口南側,大安區,2024-06-17 18:58:15,敦化南路二段293巷口南側,14,0,14,121.54899,25.02370,True
69,安和路二段181巷口,大安區,2024-06-17 19:14:20,安和路二段185號,17,0,17,121.54988,25.02721,True
72,台北科技大學億光大樓,大安區,2024-06-17 18:58:15,忠孝東路三段81號,45,0,45,121.53833,25.04177,True
76,復興南路1段340巷口,大安區,2024-06-17 19:09:14,復興南路一段信義路三段147巷45弄口,50,0,50,121.54348,25.03701,True
88,昌隆公園,大安區,2024-06-17 19:15:16,市民大道三段206號西側,16,0,16,121.53946,25.04448,True
98,瑠公公園,大安區,2024-03-30 08:59:14,忠孝東路四段49巷4弄/大安路口,20,0,20,121.54612,25.04201,False
